#Precios semanales de gasolina y diesel en USA#

Dataset en: https://www.kaggle.com/mruanova/us-gasoline-and-diesel-retail-prices-19952021

In [1]:
ruta = "/content/drive/MyDrive/DataPracticas/Combustible/PET_PRI_GND_DCUS_NUS_W.csv"

**Librerias**

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline

Significado del nombre de las columnas

The data contains the following information:

*   A1: Weekly U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon)
*   A2: Weekly U.S. All Grades Conventional Retail Gasoline Prices (Dollars per Gallon)
*   A3: Weekly U.S. All Grades Reformulated Retail Gasoline Prices (Dollars per Gallon)
*   R1: Weekly U.S. Regular All Formulations Retail Gasoline Prices (Dollars per Gallon)
*   R2: Weekly U.S. Regular Conventional Retail Gasoline Prices (Dollars per Gallon)
*   R3: Weekly U.S. Regular Reformulated Retail Gasoline Prices (Dollars per Gallon)
*   M1: Weekly U.S. Midgrade All Formulations Retail Gasoline Prices (Dollars per Gallon)
*   M2: Weekly U.S. Midgrade Conventional Retail Gasoline Prices (Dollars per Gallon)
*   M3: Weekly U.S. Midgrade Reformulated Retail Gasoline Prices (Dollars per Gallon)
*   P1: Weekly U.S. Premium All Formulations Retail Gasoline Prices (Dollars per Gallon)
*   P2: Weekly U.S. Premium Conventional Retail Gasoline Prices (Dollars per Gallon)
*   P3: Weekly U.S. Premium Reformulated Retail Gasoline Prices (Dollars per Gallon)
*   D1: Weekly U.S. No 2 Diesel Retail Prices (Dollars per Gallon)

Revision del dataset

In [3]:
df = pd.read_csv(ruta)
combustible = df.copy()
combustible = combustible.drop(combustible.loc[combustible['Date'].str.endswith("2021")].index)
combustible.head()

,Date,A1,A2,A3,R1,R2,R3,M1,M2,M3,P1,P2,P3,D1
0,01/02/1995,1.127,1.104,1.231,1.079,1.063,1.167,1.170,1.159,1.298,1.272,1.250,1.386,1.104
1,01/09/1995,1.134,1.111,1.232,1.086,1.070,1.169,1.177,1.164,1.300,1.279,1.256,1.387,1.102
2,01/16/1995,1.126,1.102,1.231,1.078,1.062,1.169,1.168,1.155,1.299,1.271,1.249,1.385,1.100
3,01/23/1995,1.132,1.110,1.226,1.083,1.068,1.165,1.177,1.165,1.296,1.277,1.256,1.378,1.095
4,01/30/1995,1.131,1.109,1.221,1.083,1.068,1.162,1.176,1.163,1.291,1.275,1.255,1.370,1.090


Revision de valores en el dataset

In [4]:
combustible.dtypes

Date     object
A1      float64
A2      float64
A3      float64
R1      float64
R2      float64
R3      float64
M1      float64
M2      float64
M3      float64
P1      float64
P2      float64
P3      float64
D1      float64
dtype: object

El dataset contiene 1357 registros de 13 tipos de combustible y una columna para la fecha en la que fue tomada la muestra

In [5]:
combustible.shape

(1357, 14)

Se agrega una columna unicamente para agregar el año y asi pueda ser segmentado el dataset

In [6]:
combustible["Year"] = combustible["Date"].apply(lambda x : x[-4:])
combustible.head()

,Date,A1,A2,A3,R1,R2,R3,M1,M2,M3,P1,P2,P3,D1,Year
0,01/02/1995,1.127,1.104,1.231,1.079,1.063,1.167,1.170,1.159,1.298,1.272,1.250,1.386,1.104,1995
1,01/09/1995,1.134,1.111,1.232,1.086,1.070,1.169,1.177,1.164,1.300,1.279,1.256,1.387,1.102,1995
2,01/16/1995,1.126,1.102,1.231,1.078,1.062,1.169,1.168,1.155,1.299,1.271,1.249,1.385,1.100,1995
3,01/23/1995,1.132,1.110,1.226,1.083,1.068,1.165,1.177,1.165,1.296,1.277,1.256,1.378,1.095,1995
4,01/30/1995,1.131,1.109,1.221,1.083,1.068,1.162,1.176,1.163,1.291,1.275,1.255,1.370,1.090,1995


Utilizando interact se crea un grafico con los cambios que ha tenido el precio durante el año

In [7]:
@interact(columna = combustible.columns.to_list()[1:-1])
def precios_por_ano(columna):
  temp = combustible.groupby(combustible["Year"])[columna].sum()
  temp = temp.drop(columns = "Date")
  temp = temp.to_frame().reset_index()
  plt.plot(temp["Year"], temp[columna])
  plt.xlabel("Years")
  plt.xticks(rotation = 90)
  plt.ylabel(str(columna) + " (USD)")
  plt.title("Prices by year for " + str(columna) + " Gasoline")
  plt.grid()
  plt.show()

interactive(children=(Dropdown(description='columna', options=('A1', 'A2', 'A3', 'R1', 'R2', 'R3', 'M1', 'M2',…

Se copia y modifica el dataset para eliminar la columna año y se transforma la columna año en int

In [8]:
new_combustible = combustible.drop(columns = "Year")
new_combustible["Date"] = new_combustible["Date"].apply(lambda x : x.replace("/", ""))
new_combustible["Date"] = new_combustible["Date"].astype("int")
new_combustible.dtypes

Date      int64
A1      float64
A2      float64
A3      float64
R1      float64
R2      float64
R3      float64
M1      float64
M2      float64
M3      float64
P1      float64
P2      float64
P3      float64
D1      float64
dtype: object

Se crea una lista con varios estimadores lineales, los cuales seran comparados

In [9]:
r_lasso = Lasso(normalize = True)
r_ridge = Ridge(normalize = True)
r_elastic = ElasticNet(normalize = True)
r_bagging = BaggingRegressor()
r_adaboost = AdaBoostRegressor(n_estimators = 10)
r_gradient = GradientBoostingRegressor(n_estimators = 10)
r_forest = RandomForestRegressor(n_estimators = 10)
estimadores = [r_lasso, r_ridge, r_elastic, r_bagging, r_adaboost, r_gradient, r_forest]

Se crea una funcion para eliminar algunos caracteres extras para poder nombrar cada uno de los estimadores en una tabla

In [10]:
def nombrar_estimadores(estimador):
  iterador = 0
  nombre = ""
  for i in str(estimador):
    iterador += 1
    if i == "(":
      nombre = str(estimador)[0:iterador - 1]
  return nombre

Se crean metricas de evaluacion

In [11]:
def rmse(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred))

def rmse_cv(estimador, X, y):
  preds = estimador.predict(X)
  return rmse(y, preds)

Se crea una funcion para hacer validacion cruzada de cada una de las columnas del dataset el valor X es la columna "Date" anteriormente transformada en int

In [12]:
def seleccion_regresor(estimador):
  resultados = []
  for i in new_combustible.columns[1:]:
    resultados.append(
        cross_val_score(
            estimador,
            new_combustible["Date"].values.reshape(-1, 1),
            new_combustible[i],
            scoring = rmse_cv,
            cv = 5
        ).mean()
    )
  return resultados

Se crea el dataframe iterando por cada uno de los estimadores para posteriormente ser evaluados

In [13]:
def crear_dataframe(estimadores):
  totales = pd.DataFrame(index = new_combustible.drop(columns = "Date").columns)
  for i in estimadores:
    totales[nombrar_estimadores(i)] = seleccion_regresor(i)
  return totales

Como se puede notar, cada uno de los estimadores dio diferentes resultados para cada columna

In [14]:
resultados = crear_dataframe(estimadores)
resultados

,Lasso,Ridge,ElasticNet,BaggingRegressor,AdaBoostRegressor,GradientBoostingRegressor,RandomForestRegressor
A1,0.966968,0.966842,0.966968,0.946526,0.935349,0.962335,0.942133
A2,0.946005,0.945874,0.946005,0.929625,0.913096,0.941967,0.934182
A3,1.001862,1.001752,1.001862,0.941139,0.965891,0.996004,0.937833
R1,0.953016,0.952905,0.953016,0.942650,0.917531,0.948577,0.929630
R2,0.933939,0.933810,0.933939,0.931569,0.900244,0.929852,0.932653
R3,0.989760,0.989688,0.989760,0.946841,0.951764,0.984301,0.945287
M1,1.009237,1.009036,1.009237,0.920192,0.972719,1.003328,0.927232
M2,0.977261,0.977064,0.977261,0.929052,0.945276,0.972581,0.930290
M3,1.045017,1.044827,1.045017,0.948972,1.017594,1.037707,0.940874
P1,1.052161,1.051920,1.052161,0.949993,1.013581,1.045918,0.952197


Utilizando de nuevo interact se puede seleccionar una columna del dataset anterior y comparar los resultados de cada uno de los estimadores, por ultimo, se agrego una nueva columna que indica que estimador fue el mejor

In [15]:
@interact(combustible = new_combustible.drop(columns = "Date").columns)
def mejor_estimador(combustible):
  valor = resultados[resultados.index == combustible]
  valor["Mejor estimador"] = valor.idxmin(axis = 1)
  return valor

interactive(children=(Dropdown(description='combustible', options=('A1', 'A2', 'A3', 'R1', 'R2', 'R3', 'M1', '…